# Pre-processing to translate Portuguese into Korean
- Result: `./eda/review/artifact/reviews_preprocessed.csv`

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../../downloads/olist/olist_order_reviews_dataset.csv")
df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [3]:
contents = df[['review_comment_title', "review_comment_message"]].dropna(how='all')
contents = df.loc[contents.index]
contents = contents[['review_id', 'review_comment_title', 'review_comment_message']]
contents

,review_id,review_comment_title,review_comment_message
3,e64fb393e7b32834bb789ff8bb30750e,NaN,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,NaN,Parabéns lojas lannister adorei comprar pela I...
9,8670d52e15e00043ae7de4c01cc2fe06,recomendo,aparelho eficiente. no site a marca do aparelh...
12,4b49719c8a200003f700d3d986ea1a19,NaN,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
15,3948b09f7c818e2d86c9a546758b2335,Super recomendo,"Vendedor confiável, produto ok e entrega antes..."
...,...,...,...
99205,98fffa80dc9acbde7388bef1600f3b15,NaN,para este produto recebi de acordo com a compr...
99208,df5fae90e85354241d5d64a8955b2b09,NaN,Entregou dentro do prazo. O produto chegou em ...
99215,a709d176f59bc3af77f4149c96bae357,NaN,"O produto não foi enviado com NF, não existe v..."
99221,b3de70c89b1510c4cd3d0649fd302472,NaN,"Excelente mochila, entrega super rápida. Super..."


In [4]:
contents['preprocessed_comment'] = contents['review_comment_message'].str.replace('\r\n', ' ')
contents['preprocessed_comment'] = contents['preprocessed_comment'].str.replace('\n', ' ')
contents['preprocessed_comment'] = contents['preprocessed_comment'].str.replace('\r', ' ')
contents['preprocessed_comment'] = contents['preprocessed_comment'].str.strip()
contents['preprocessed_title'] = contents['review_comment_title'].str.replace('\r\n', ' ')
contents['preprocessed_title'] = contents['preprocessed_title'].str.replace('\n', ' ')
contents['preprocessed_title'] = contents['preprocessed_title'].str.replace('\r', ' ')
contents['preprocessed_title'] = contents['preprocessed_title'].str.strip()

import os
os.makedirs("./artifact", exist_ok=True)
contents[['review_id', 'preprocessed_title', 'preprocessed_comment']].to_csv('./artifact/reviews_preprocessed.csv')